In [1]:
from together import Together

client = Together()

response = client.chat.completions.create(
    model="deepseek-ai/DeepSeek-R1-Distill-Llama-70B-free",
    messages=[
      {
        "role": "user",
        "content": "What are some fun things to do in New York?"
      }
    ]
)
print(response.choices[0].message.content)

<think>
Okay, so I need to figure out some fun things to do in New York. I'm not super familiar with the city, but I know it's a big place with lots to offer. Let me start by brainstorming what I know and then maybe add some ideas.

First off, I remember hearing about the Statue of Liberty. That's a major landmark, right? I think you can take a ferry to Liberty Island to see it up close. There's also Ellis Island nearby, which I think is a museum about immigration. That could be interesting from a historical perspective.

Then there's Central Park. I've seen it in movies, like when people ice skate or have picnics. It's a big park in the middle of Manhattan, so it's probably a nice place to relax or take a walk. Maybe there are some attractions inside the park too, like a zoo or a lake where you can rent boats.

Times Square is another place I've heard about. It's super busy with all the lights and billboards. I think people go there for the atmosphere, maybe to shop or catch a Broadwa

In [2]:
import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [3]:
from minsearch import AppendableIndex

index = AppendableIndex(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

index.fit(documents)

In [6]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5,
        output_ids=True
    )

    return results

In [6]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

<QUESTION>
{question}
</QUESTION>

<CONTEXT>
{context}
</CONTEXT>
""".strip()

def build_prompt(query, search_results):
    context = ""

    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    # print(prompt)
    return prompt

In [7]:
from together import Together

client = Together()
def llm(prompt):
    response = client.chat.completions.create(
        model= 'meta-llama/Llama-3.3-70B-Instruct-Turbo-Free',
        # model="deepseek-ai/DeepSeek-R1-Distill-Llama-70B-free",
        messages=[
        {
            "role": "user",
            "content": prompt
        }
        ],
        # response_format='json'
    )
    return response.choices[0].message.content

In [8]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    # print(prompt)
    answer = llm(prompt)
    return answer

In [9]:
print(rag(query="Can i get certificate if I join the course now?"))

Yes, you can still join the course after the start date and you are eligible to submit homework. However, to get a certificate, you need to finish the course with a "live" cohort, which implies that you should join the course while it is still running, as certificates are not awarded for the self-paced mode.


## Agentic Rag

In [10]:
prompt_template = """
You're a course teaching assistant.

You're given a QUESTION from a course student and that you need to answer with your own knowledge and provided CONTEXT.
At the beginning the context is EMPTY.

You MUST respond in one of the three EXACT JSON templates shown below.

Do NOT include any explanations, thoughts, or text outside the JSON block.

<QUESTION>
{question}
</QUESTION>

<CONTEXT> 
{context}
</CONTEXT>

If CONTEXT is EMPTY, you can use our FAQ database.
In this case, use the following output template:

{{
"action": "SEARCH",
"reasoning": "<add your reasoning here>"
}}

If you can answer the QUESTION using CONTEXT, use this template:

{{
"action": "ANSWER",
"answer": "<your answer>",
"source": "CONTEXT"
}}

If the context doesn't contain the answer, use your own knowledge to answer the question

{{
"action": "ANSWER",
"answer": "<your answer>",
"source": "OWN_KNOWLEDGE"
}}
""".strip()

In [11]:
question = "how do I join the course now?"
context = "EMPTY"

prompt = prompt_template.format(question=question, context=context)
# print(prompt)

answer = llm(prompt)
print(answer)

{
"action": "SEARCH",
"reasoning": "The context is empty, so I need to search the FAQ database to find out how to join the course."
}


In [12]:
def build_context(search_results):
    context = ""

    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

    return context.strip()

search_results = search(question)
context = build_context(search_results)
prompt = prompt_template.format(question=question, context=context)
print(prompt)

You're a course teaching assistant.

You're given a QUESTION from a course student and that you need to answer with your own knowledge and provided CONTEXT.
At the beginning the context is EMPTY.

You MUST respond in one of the three EXACT JSON templates shown below.

Do NOT include any explanations, thoughts, or text outside the JSON block.

<QUESTION>
how do I join the course now?
</QUESTION>

<CONTEXT> 
section: General course-related questions
question: Course - Can I still join the course after the start date?
answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

section: General course-related questions
question: Course - When will the course start?
answer: The purpose of this document is to capture frequently asked technical questions
The exact day and hour of the course will be 15th Jan 2024 at 17h00. The course wil

In [13]:
answer = llm(prompt)
print(answer)

{
"action": "ANSWER",
"answer": "To join the course, you should register before the course starts using the provided link, subscribe to the course public Google Calendar, and join the course Telegram channel with announcements. Don't forget to register in DataTalks.Club's Slack and join the channel.",
"source": "CONTEXT"
}


In [14]:
import json
def agentic_rag_v1(question):
    context = "EMPTY"
    prompt = prompt_template.format(question=question, context=context)
    answer_json = llm(prompt)
    answer = json.loads(answer_json)
    print(answer)

    if answer['action'] == 'SEARCH':
        print('need to perform search...')
        search_results = search(question)
        context = build_context(search_results)
        
        prompt = prompt_template.format(question=question, context=context)
        answer_json = llm(prompt)
        answer = json.loads(answer_json)
        print(answer)

    return answer

In [15]:
agentic_rag_v1(question="How can i run terraform in GCP?")
agentic_rag_v1(question="How can i join the course")

{'action': 'ANSWER', 'answer': "To run Terraform in GCP, you can use the Google Cloud Shell or install Terraform on a GCP VM. First, install Terraform and the Google Cloud SDK. Then, set up your GCP credentials and configure your Terraform configuration files to use the GCP provider. Finally, run 'terraform init', 'terraform plan', and 'terraform apply' to deploy your infrastructure.", 'source': 'OWN_KNOWLEDGE'}
{'action': 'SEARCH', 'reasoning': 'The context is empty, so I need to search the FAQ database to find the steps to join the course.'}
need to perform search...
{'action': 'ANSWER', 'answer': 'You can join the course by registering before the course starts using the provided link, and then join the course Telegram channel with announcements. You can also start learning and submitting homework without registering, as registration is just to gauge interest before the start date.', 'source': 'CONTEXT'}


{'action': 'ANSWER',
 'answer': 'You can join the course by registering before the course starts using the provided link, and then join the course Telegram channel with announcements. You can also start learning and submitting homework without registering, as registration is just to gauge interest before the start date.',
 'source': 'CONTEXT'}

## Agentic search

In [21]:
prompt_template = """
You're a course teaching assistant.

You're given a QUESTION from a course student and that you need to answer with your own knowledge and provided CONTEXT.

You MUST respond in one of the three EXACT JSON templates shown below.

Do NOT include any explanations, thoughts, or text outside the JSON block.

The CONTEXT is build with the documents from our FAQ database.
SEARCH_QUERIES contains the queries that were used to retrieve the documents
from FAQ to and add them to the context.
PREVIOUS_ACTIONS contains the actions you already performed.

At the beginning the CONTEXT is empty.

You can perform the following actions:

- Search in the FAQ database to get more data for the CONTEXT
- Answer the question using the CONTEXT
- Answer the question using your own knowledge

For the SEARCH action, build search requests based on the CONTEXT and the QUESTION.
Carefully analyze the CONTEXT and generate the requests to deeply explore the topic. 

Don't use search queries used at the previous iterations.

Don't repeat previously performed actions.

Don't perform more than {max_iterations} iterations for a given student question.
The current iteration number: {iteration_number}. If we exceed the allowed number 
of iterations, give the best possible answer with the provided information.

Output templates:

If you want to perform search, use this template:

{{
"action": "SEARCH",
"reasoning": "<add your reasoning here>",
"keywords": ["search query 1", "search query 2", ...]
}}

If you can answer the QUESTION using CONTEXT, use this template:

{{
"action": "ANSWER_CONTEXT",
"answer": "<your answer>",
"source": "CONTEXT"
}}

If the context doesn't contain the answer, use your own knowledge to answer the question

{{
"action": "ANSWER",
"answer": "<your answer>",
"source": "OWN_KNOWLEDGE"
}}

<QUESTION>
{question}
</QUESTION>

<SEARCH_QUERIES>
{search_queries}
</SEARCH_QUERIES>

<CONTEXT> 
{context}
</CONTEXT>

<PREVIOUS_ACTIONS>
{previous_actions}
</PREVIOUS_ACTIONS>
""".strip()

In [22]:
question = "how do I join the course?"

search_queries = []
search_results = []
previous_actions = []
context = build_context(search_results)

prompt = prompt_template.format(
    question=question,
    context=context,
    search_queries="\n".join(search_queries),
    previous_actions='\n'.join([json.dumps(a) for a in previous_actions]),
    max_iterations=3,
    iteration_number=1
)
print(prompt)

You're a course teaching assistant.

You're given a QUESTION from a course student and that you need to answer with your own knowledge and provided CONTEXT.

You MUST respond in one of the three EXACT JSON templates shown below.

Do NOT include any explanations, thoughts, or text outside the JSON block.

The CONTEXT is build with the documents from our FAQ database.
SEARCH_QUERIES contains the queries that were used to retrieve the documents
from FAQ to and add them to the context.
PREVIOUS_ACTIONS contains the actions you already performed.

At the beginning the CONTEXT is empty.

You can perform the following actions:

- Search in the FAQ database to get more data for the CONTEXT
- Answer the question using the CONTEXT
- Answer the question using your own knowledge

For the SEARCH action, build search requests based on the CONTEXT and the QUESTION.
Carefully analyze the CONTEXT and generate the requests to deeply explore the topic. 

Don't use search queries used at the previous iter

In [23]:
answer_json = llm(prompt)
answer = json.loads(answer_json)
print(json.dumps(answer, indent=2))

{
  "action": "SEARCH",
  "reasoning": "The question is about joining a course, so I need to search for relevant information in the FAQ database to provide a helpful answer. I'll use search queries related to course enrollment and registration.",
  "keywords": [
    "course enrollment",
    "registration process",
    "joining a course"
  ]
}


In [26]:
previous_actions.append(answer)

In [27]:
keywords = answer['keywords']
search_queries.extend(keywords)

# And perform the search:

for k in keywords:
    res = search(k)
    search_results.extend(res)

In [4]:
def dedup(seq):
    seen = set()
    result = []
    for el in seq:
        _id = el['_id']
        if _id in seen:
            continue
        seen.add(_id)
        result.append(el)
    return result

In [ ]:
search_results = dedup(search_results)

In [29]:
context = build_context(search_results)

prompt = prompt_template.format(
    question=question,
    context=context,
    search_queries="\n".join(search_queries),
    previous_actions='\n'.join([json.dumps(a) for a in previous_actions]),
    max_iterations=3,
    iteration_number=2
)
print(prompt)

answer_json = llm(prompt)
answer = json.loads(answer_json)
print(json.dumps(answer, indent=2))

You're a course teaching assistant.

You're given a QUESTION from a course student and that you need to answer with your own knowledge and provided CONTEXT.

You MUST respond in one of the three EXACT JSON templates shown below.

Do NOT include any explanations, thoughts, or text outside the JSON block.

The CONTEXT is build with the documents from our FAQ database.
SEARCH_QUERIES contains the queries that were used to retrieve the documents
from FAQ to and add them to the context.
PREVIOUS_ACTIONS contains the actions you already performed.

At the beginning the CONTEXT is empty.

You can perform the following actions:

- Search in the FAQ database to get more data for the CONTEXT
- Answer the question using the CONTEXT
- Answer the question using your own knowledge

For the SEARCH action, build search requests based on the CONTEXT and the QUESTION.
Carefully analyze the CONTEXT and generate the requests to deeply explore the topic. 

Don't use search queries used at the previous iter

In [32]:
def agentic_search(question):
    search_queries = []
    search_results = []
    previous_actions = []

    iteration = 0
    
    while True:
        print(f'ITERATION #{iteration}...')
    
        context = build_context(search_results)
        prompt = prompt_template.format(
            question=question,
            context=context,
            search_queries="\n".join(search_queries),
            previous_actions='\n'.join([json.dumps(a) for a in previous_actions]),
            max_iterations=3,
            iteration_number=iteration
        )
    
        # print(prompt)
    
        answer_json = llm(prompt)
        answer = json.loads(answer_json)
        # print(json.dumps(answer, indent=2))

        previous_actions.append(answer)
    
        action = answer['action']
        if action != 'SEARCH':
            break
    
        keywords = answer['keywords']
        search_queries = list(set(search_queries) | set(keywords))

        for k in keywords:
            res = search(k)
            search_results.extend(res)
    
        search_results = dedup(search_results)
        
        iteration = iteration + 1
        if iteration >= 4:
            break
    
        print()

    return answer

In [33]:
print(agentic_search(question="How can i do well in Module 1 of the course?"))

ITERATION #0...

ITERATION #1...
{'action': 'ANSWER', 'answer': "To do well in Module 1 of the course, it's essential to have a good understanding of the prerequisites and syllabus. Make sure you have installed and set up all the dependencies and requirements, including a Google cloud account, Google Cloud SDK, Python 3, Terraform, and Git. It's also crucial to look over the course outline and requirements for Module 1, which includes Docker and Terraform. You can find more specific information and tips in the CONTEXT provided, such as solutions to common errors like ModuleNotFoundError for psycopg2 or py4j. Additionally, following the course materials, participating in discussions, and completing assignments on time will help you stay on track and perform well in the module.", 'source': 'OWN_KNOWLEDGE'}


## Function calling

In [5]:
search_tool = {
    "type": "function",
    "function":{
    "name": "search",
    "description": "Search the FAQ database",
    "parameters": {
        "type": "object",
        "properties": {
            "query": {
                "type": "string",
                "description": "Search query text to look up in the course FAQ."
            }
        },
        "required": ["query"],
        "additionalProperties": False
    }
}
}

In [97]:
question = "How can i do well in Module 1 of the course?"

developer_prompt = """
You're a course teaching assistant. 
You're given a question from a course student and your task is to answer it.
""".strip()

tools = [search_tool]

chat_messages = [
    {"role": "system", "content": developer_prompt},
    {"role": "user", "content": question}
]
# client = Together()
response = client.chat.completions.create(
        model= 'meta-llama/Llama-3.3-70B-Instruct-Turbo-Free',
        messages=chat_messages,
        tools=tools
)
print(response)

id='o3ZUyM8-4Yz4kd-9611d4b81a7d0e6e' object=<ObjectType.ChatCompletion: 'chat.completion'> created=1752840156 model='meta-llama/Llama-3.3-70B-Instruct-Turbo-Free' choices=[ChatCompletionChoicesData(index=0, logprobs=None, seed=11789080685737837000, finish_reason=<FinishReason.ToolCalls: 'tool_calls'>, message=ChatCompletionMessage(role=<MessageRole.ASSISTANT: 'assistant'>, content=None, tool_calls=[ToolCalls(id='call_yvghg0k0zdu1nv7jw5ezzaac', type='function', function=FunctionCall(name='search', arguments='{"query":"How can I do well in Module 1 of the course?"}'), index=0)]))] prompt=[] usage=UsageData(prompt_tokens=231, completion_tokens=28, total_tokens=259, cached_tokens=0)


In [ ]:
response.choices[0].message.tool_calls

ChatCompletionMessage(role=<MessageRole.ASSISTANT: 'assistant'>, content=None, tool_calls=[ToolCalls(id='call_yvghg0k0zdu1nv7jw5ezzaac', type='function', function=FunctionCall(name='search', arguments='{"query":"How can I do well in Module 1 of the course?"}'), index=0)])

In [82]:
response.choices[0].message.tool_calls[0]

ToolCalls(id='call_oswxzguu9tvat9w7xkgw5ffg', type='function', function=FunctionCall(name='search', arguments='{"query":"How can I do well in Module 1 of the course?"}'), index=0)

In [99]:
calls = response.choices[0].message.tool_calls
call = calls[0]
call

call_id = call.id
call_id

f_name = call.function.name
f_name

arguments = json.loads(call.function.arguments)
arguments

{'query': 'How can I do well in Module 1 of the course?'}

In [100]:
f = globals()[f_name]

In [101]:
results = f(**arguments)
search_results = json.dumps(results, indent=2)
print(search_results)

[
  {
    "text": "No, you can only get a certificate if you finish the course with a \u201clive\u201d cohort. We don't award certificates for the self-paced mode. The reason is you need to peer-review capstone(s) after submitting a project. You can only peer-review projects at the time the course is running.",
    "section": "General course-related questions",
    "question": "Certificate - Can I follow the course in a self-paced mode and get a certificate?",
    "course": "data-engineering-zoomcamp",
    "_id": 11
  },
  {
    "text": "Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.",
    "section": "General course-related questions",
    "question": "Course - Can I follow the course after it finishes?",
    "course": "data-engineering-zoomcamp",
  

In [102]:
chat_messages

[{'role': 'system',
  'content': "You're a course teaching assistant. \nYou're given a question from a course student and your task is to answer it."},
 {'role': 'user', 'content': 'How can i do well in Module 1 of the course?'}]

In [103]:
# chat_messages.append(call)

chat_messages.append({
    "role": "tool",
    "tool_call_id": call.id,
    "name": f_name,
    "content": search_results,
})

In [104]:
response = client.chat.completions.create(
        model= 'meta-llama/Llama-3.3-70B-Instruct-Turbo-Free',
        messages=chat_messages,
        tools=tools
)

In [ ]:
print(response.choices[0].message.content)

role=<MessageRole.ASSISTANT: 'assistant'> content="To do well in Module 1 of the course, here are some general tips that can be applied:\n\n1. **Start Early**: Begin your coursework as soon as possible to avoid last-minute rushes. This allows you to absorb the material better and manage your time more effectively.\n\n2. **Read and Understand the Course Material**: Make sure you read through all the provided resources, including textbooks, articles, and online resources. Understanding the course material is key to performing well.\n\n3. **Participate in Discussions**: Engage with your peers and instructors through discussion forums or live sessions. This can help clarify any doubts you have and provide additional insights into the course material.\n\n4. **Complete Assignments and Homework**: These are crucial for understanding the practical application of the concepts learned. Ensure you complete all assignments on time and to the best of your ability.\n\n5. **Seek Help When Needed**: D

In [115]:
developer_prompt = """
You're a course teaching assistant. 
You're given a question from a course student and your task is to answer it.
If you look up something in FAQ, convert the student question into multiple queries.
""".strip()

chat_messages = [
    {"role": "system", "content": developer_prompt},
    {"role": "user", "content": question}
]

response = client.chat.completions.create(
        model= 'meta-llama/Llama-3.3-70B-Instruct-Turbo-Free',
        messages=chat_messages,
        tools=tools
)

In [14]:
import json
def do_call(tool_call_response):
    function_name = tool_call_response.function.name
    arguments = json.loads(tool_call_response.function.arguments)

    f = globals()[function_name]
    result = f(**arguments)

    return {
    "role": "tool",
    "tool_call_id": tool_call_response.id,
    "name": function_name,
    "content": json.dumps(result, indent=2),
    }

In [116]:
response.choices[0]

ChatCompletionChoicesData(index=0, logprobs=None, seed=17004802788733580000, finish_reason=<FinishReason.ToolCalls: 'tool_calls'>, message=ChatCompletionMessage(role=<MessageRole.ASSISTANT: 'assistant'>, content=None, tool_calls=[ToolCalls(id='call_ouzspzrtemlk351b8vrpzq5l', type='function', function=FunctionCall(name='search', arguments='{"query":"How can I do well in Module 1 of the course?"}'), index=0)]))

In [117]:
for entry in response.choices:

    if entry.message.tool_calls != []:      
        result = do_call(entry.message.tool_calls[0])
        chat_messages.append(result)
    else:
        print(entry.message.content) 

In [118]:
chat_messages

[{'role': 'system',
  'content': "You're a course teaching assistant. \nYou're given a question from a course student and your task is to answer it.\nIf you look up something in FAQ, convert the student question into multiple queries."},
 {'role': 'user', 'content': 'How can i do well in Module 1 of the course?'},
 {'role': 'tool',
  'tool_call_id': 'call_ouzspzrtemlk351b8vrpzq5l',
  'name': 'search',
  'content': '[\n  {\n    "text": "No, you can only get a certificate if you finish the course with a \\u201clive\\u201d cohort. We don\'t award certificates for the self-paced mode. The reason is you need to peer-review capstone(s) after submitting a project. You can only peer-review projects at the time the course is running.",\n    "section": "General course-related questions",\n    "question": "Certificate - Can I follow the course in a self-paced mode and get a certificate?",\n    "course": "data-engineering-zoomcamp",\n    "_id": 11\n  },\n  {\n    "text": "Yes, we will keep all the

In [119]:
response = client.chat.completions.create(
        model= 'meta-llama/Llama-3.3-70B-Instruct-Turbo-Free',
        messages=chat_messages,
        tools=tools
)
for entry in response.choices:

    if entry.message.tool_calls != []:      
        result = do_call(entry.message.tool_calls[0])
        chat_messages.append(result)
    else:
        print(entry.message.content) 

To do well in Module 1 of the course, here are some steps you can follow:

1. **Read the course materials**: Start by reading the course materials, including the lecture notes, slides, and any additional resources provided.
2. **Watch the video lectures**: Watch the video lectures for Module 1, taking notes on key concepts and ideas.
3. **Complete the homework assignments**: Complete the homework assignments for Module 1, which will help you practice and reinforce your understanding of the material.
4. **Participate in discussions**: Participate in discussions on the course forum or Slack channel, asking questions and engaging with your peers and instructors.
5. **Review and practice**: Review the material regularly, practicing what you've learned and seeking help when needed.

Additionally, you can also:

* **Star the repo**: Star the course repository on GitHub to show your support and interest in the course.
* **Contribute to the course**: Consider contributing to the course by crea

In [122]:
developer_prompt = """
You're a course teaching assistant. 
You're given a question from a course student and your task is to answer it.

Use FAQ if your own knowledge is not sufficient to answer the question.
When using FAQ, perform deep topic exploration: make one request to FAQ,
and then based on the results, make more requests.

At the end of each response, ask the user a follow up question based on your answer.
""".strip()

chat_messages = [
    {"role": "system", "content": developer_prompt},
]

In [124]:
while True: # main Q&A loop
    question = input() # How do I do my best for module 1?
    if question == 'stop':
        break

    message = {"role": "user", "content": question}
    chat_messages.append(message)

    while True: # request-response loop - query API till get a message
        response = client.chat.completions.create(
        model= 'meta-llama/Llama-3.3-70B-Instruct-Turbo-Free',
        messages=chat_messages,
        tools=tools
        )

        has_messages = False
        for entry in response.choices:

            if entry.message.tool_calls != []: 
                print('function_call:', entry.message.tool_calls[0])
                print()    
                result = do_call(entry.message.tool_calls[0])
                chat_messages.append(result)
            else:
                print(entry.message.content) 
                print()
                has_messages = True

        if has_messages:
            break

function_call: id='call_lc9httvuxe48txprcbl9ke2z' type='function' function=FunctionCall(name='search', arguments='{"query":"joining the course"}') index=0

To join the course, you should register before the course starts using the provided link, subscribe to the course public Google Calendar, join the course Telegram channel with announcements, and register in DataTalks.Club's Slack and join the channel.

What is your current level of experience with data engineering, and are you looking to learn specific skills or topics in the course?



In [16]:
import markdown
def display_function_call( entry, result):
        call_html = f"""
            <details>
            <summary>Function call: <tt>{entry.function.name}({(entry.function.arguments)})</tt></summary>
            <div>
                <b>Call</b>
                <pre>{entry}</pre>
            </div>
            <div>
                <b>Output</b>
                <pre>{result.message.content}</pre>
            </div>
            
            </details>
        """
        display(HTML(call_html))

def display_response( entry):
    response_html = markdown.markdown(entry.message.content)
    html = f"""
        <div>
            <div><b>Assistant:</b></div>
            <div>{response_html}</div>
        </div>
    """
    display(HTML(html))

In [17]:
from IPython.display import display, HTML
import markdown # pip install markdown

    

developer_prompt = """
You're a course teaching assistant. 
You're given a question from a course student and your task is to answer it.

Use FAQ if your own knowledge is not sufficient to answer the question.

At the end of each response, ask the user a follow up question based on your answer.
""".strip()
tools = [search_tool]
chat_messages = [
    {"role": "system", "content": developer_prompt},
]

# Chat loop
while True:
    question = input()
    if question.strip().lower() == 'stop':
        print("Chat ended.")
        break
    print()

    message = {"role": "user", "content": question}
    chat_messages.append(message)

    while True:  # inner request loop
        response = client.chat.completions.create(
        model= 'meta-llama/Llama-3.3-70B-Instruct-Turbo-Free',
        messages=chat_messages,
        tools=tools
        )

        has_messages = False

        for entry in response.choices:
            if entry.message.tool_calls != []:   
                result = do_call(entry.message.tool_calls[0])
                chat_messages.append(result)
                display_function_call(entry.message.tool_calls[0], result)
            else:
                display_response(entry)
                has_messages = True

        if has_messages:
            break

AttributeError: 'dict' object has no attribute 'message'

In [ ]:
def add_entry(question, answer):
    doc = {
        'question': question,
        'text': answer,
        'section': 'user added',
        'course': 'data-engineering-zoomcamp'
    }
    index.append(doc)

add_entry_description = {
    "type": "function",
    "function": {
    "name": "add_entry",
    "description": "Add an entry to the FAQ database",
    "parameters": {
        "type": "object",
        "properties": {
            "question": {
                "type": "string",
                "description": "The question to be added to the FAQ database",
            },
            "answer": {
                "type": "string",
                "description": "The answer to the question",
            }
        },
        "required": ["question", "answer"],
        "additionalProperties": False
    }
}
}